In [1]:
import os
import pandas as pd
import numpy as np

### Define the path to the directory containing the cleaned CSV files.

In [295]:
dataset_path = "olist_dataset_cleaned"

# Load each dataset by its specific filename for clarity and reliability.
# This prevents errors caused by inconsistent file system ordering.
olist_customers_dataset = pd.read_csv(os.path.join(dataset_path, "olist_customers_dataset.csv"))
olist_geolocation_dataset = pd.read_csv(os.path.join(dataset_path, "olist_geolocation_dataset.csv"))
olist_order_dataset = pd.read_csv(os.path.join(dataset_path, "olist_orders_dataset.csv"))
olist_order_items_dataset = pd.read_csv(os.path.join(dataset_path, "olist_order_items_dataset.csv"))
olist_order_payments_dataset = pd.read_csv(os.path.join(dataset_path, "olist_order_payments_dataset.csv"))
olist_order_reviews_dataset = pd.read_csv(os.path.join(dataset_path, "olist_order_reviews_dataset.csv"))
olist_products_dataset = pd.read_csv(os.path.join(dataset_path, "olist_products_dataset.csv"))
olist_sellers_dataset = pd.read_csv(os.path.join(dataset_path, "olist_sellers_dataset.csv"))
product_category_name_translation = pd.read_csv(os.path.join(dataset_path, "product_category_name_translation.csv"))

# Optional: Print a confirmation message to verify successful loading.
print("All 9 cleaned datasets have been successfully loaded into Pandas DataFrames.")

All 9 cleaned datasets have been successfully loaded into Pandas DataFrames.


### 2. Create the Analytical Master Table
* Combine all individual DataFrames into a single, comprehensive "master" table.
* This unified view allows for holistic analysis across all aspects of an order.
* The strategy is to start with the central 'orders' table and enrich it using
* a series of 'left' joins to ensure no order information is lost in the process.

In [296]:
# Start with the core transaction data: orders and the items within them.
df_master = pd.merge(olist_order_dataset, olist_order_items_dataset, on='order_id', how='left')

# Add payment information for each order.
df_master = pd.merge(df_master, olist_order_payments_dataset, on='order_id', how='left')

# Add customer review data.
df_master = pd.merge(df_master, olist_order_reviews_dataset, on='order_id', how='left')

# Add product details for each item.
df_master = pd.merge(df_master, olist_products_dataset, on='product_id', how='left')

# Add the customer information associated with each order.
df_master = pd.merge(df_master, olist_customers_dataset, on='customer_id', how='left')

# Add the seller information for each item.
df_master = pd.merge(df_master, olist_sellers_dataset, on='seller_id', how='left')

# Add the English translation for product category names for easier analysis.
df_master = pd.merge(df_master, product_category_name_translation, on='product_category_name', how='left')

# Optional: Display the shape and columns of the final master table to verify the joins.
print("Master table created successfully.")
print(f"Shape of the master table: {df_master.shape}")
# print(f"Columns: {df_master.columns.to_list()}")

Master table created successfully.
Shape of the master table: (119143, 40)


### Data Converting and Cleaning

In [298]:
### 3. Data Type Conversion and Final Cleaning
# Prepare the master table for analysis by handling data types and removing invalid rows.

# Convert all date-related columns from string objects to datetime objects.
# This is essential for performing any time-based calculations (e.g., delivery time).
date_columns_to_convert = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'shipping_limit_date',
    'review_creation_date',
    'review_answer_timestamp'
]
for column in date_columns_to_convert:
    df_master[column] = pd.to_datetime(df_master[column], errors='coerce')

# Clean the DataFrame by removing rows with missing critical information.
# Rows are dropped if they lack an item_id, payment_type, category, or seller,
# as these represent incomplete or invalid transactions for our business analysis.
df_master.dropna(
    subset=[
        'order_item_id',
        'payment_type',
        'product_category_name',
        'seller_zip_code_prefix'
    ],
    inplace=True
)

# Filter the dataset to focus on the most relevant and complete years (2017 and 2018).
# The year 2016 contains sparse, incomplete data and is excluded to ensure
# that the analysis reflects the company's mature operational period.
df_analysis = df_master[df_master['order_purchase_timestamp'].dt.year != 2016].copy()

# Optional: Verify the cleaning by checking the final shape and remaining nulls.
print("\nData cleaning and preparation complete.")
print(f"Final shape of the analysis DataFrame (df_analysis): {df_analysis.shape}")


Data cleaning and preparation complete.
Final shape of the analysis DataFrame (df_analysis): (117898, 40)


### Data Analyzing

#### What is the Total GMV (Gross Merchandise Volume) over time?

In [299]:
# --- Section A, Question 1: What is the Total GMV (Gross Merchandise Volume) over time? ---

# BEST PRACTICE: Create a dedicated, clean DataFrame for the analysis period.
# This makes all subsequent queries simpler and avoids re-filtering.
# We include your logic to remove 2016 and the incomplete final month of 2018.
df_analysis = df_master[
    (df_master['order_purchase_timestamp'].dt.year.isin([2017, 2018])) &
    (df_master['order_purchase_timestamp'] < '2018-09-01')  # A robust way to exclude sparse final data
].copy()

# Step 1: Create the 'year_month' column for grouping.
df_analysis['year_month'] = df_analysis['order_purchase_timestamp'].dt.strftime('%Y-%m')

# Step 2: Group by month and aggregate (your logic is perfect here).
# We get the count of unique orders and the sum of payment_value.
monthly_gmv = df_analysis.groupby('year_month').agg(
    Number_of_Orders=('order_id', 'nunique'),
    Total_Revenue=('payment_value', 'sum')
).reset_index()

# Step 3: Sort chronologically to ensure the trend is clear.
monthly_gmv = monthly_gmv.sort_values(by='year_month')

# Step 4: Create a formatted column for the final report.
monthly_gmv['Total_Revenue_Formatted'] = monthly_gmv['Total_Revenue'].map('R${:,.2f}'.format)

# --- Final Display ---
print("--- 📊 Total GMV and Order Count per Month (2017 - Aug 2018) ---")
display(monthly_gmv[['year_month', 'Number_of_Orders', 'Total_Revenue_Formatted']])

--- 📊 Total GMV and Order Count per Month (2017 - Aug 2018) ---


,year_month,Number_of_Orders,Total_Revenue_Formatted
0,2017-01,788,"R$187,609.02"
1,2017-02,1733,"R$346,280.99"
2,2017-03,2638,"R$529,855.95"
3,2017-04,2385,"R$506,510.85"
4,2017-05,3660,"R$730,912.77"
5,2017-06,3215,"R$605,367.11"
6,2017-07,3967,"R$741,824.40"
7,2017-08,4291,"R$877,924.67"
8,2017-09,4243,"R$1,022,896.05"
9,2017-10,4568,"R$1,035,728.78"


My analysis of Gross Merchandise Volume (GMV) over time reveals a story of rapid growth in 2017 followed by a period of high-level stability and a slight softening in mid-2018.

*   **Explosive 2017 Growth:** The platform experienced a hyper-growth phase in 2017, with monthly GMV scaling from **R$187k** in January to consistently over **R$1 Million** by the end of the year. This demonstrates a strong and successful expansion of the business.

*   **The "Black Friday" Peak:** A significant seasonal peak occurred in **November 2017**, which generated nearly **R$1.6 Million** in GMV. This spike confirms that the Black Friday sales period is the single most important revenue event for the company.

*   **2018 Stability and Decline:** Olist successfully maintained its high performance into 2018, establishing a new, stable baseline well above R$1.2 Million monthly and reaching an all-time peak of **R$1.5 Million** in May 2018. However, this peak was followed by a consistent decline over the next three months, a critical trend that warrants further investigation into potential seasonality or external market factors.

#### Who are the top 10 best-selling product categories?

In [240]:
# --- Section A, Question 2: Top 10 Best-Selling Product Categories ---

# Use the consistent df_analysis DataFrame and group by the English category name for clarity.
category_performance = df_analysis.groupby('product_category_name_english').agg(
    Items_Sold=('order_item_id', 'count'),
    Total_Revenue=('payment_value', 'sum')
)

# Analysis by Revenue (your original method)
top_10_by_revenue = category_performance.sort_values(by='Total_Revenue', ascending=False).head(10)
top_10_by_revenue['Total_Revenue'] = top_10_by_revenue['Total_Revenue'].map('R${:,.2f}'.format)

# Analysis by Items Sold (a complementary view for a complete picture)
top_10_by_items = category_performance.sort_values(by='Items_Sold', ascending=False).head(10)
top_10_by_items['Total_Revenue'] = top_10_by_items['Total_Revenue'].map('R${:,.2f}'.format)


# --- Display Final Results ---
print("--- 👑 Top 10 Categories by Total Revenue ---")
display(top_10_by_revenue)

print("\n--- 🚀 Top 10 Categories by Items Sold ---")
display(top_10_by_items)

--- 👑 Top 10 Categories by Total Revenue ---


,Items_Sold,Total_Revenue
product_category_name_english,,
bed_bath_table,11980,"R$1,741,707.09"
health_beauty,9979,"R$1,656,901.43"
computers_accessories,8126,"R$1,597,084.36"
furniture_decor,8751,"R$1,432,399.73"
watches_gifts,6208,"R$1,427,085.44"
sports_leisure,8985,"R$1,396,295.96"
housewares,7368,"R$1,095,163.57"
auto,4388,"R$852,378.98"
garden_tools,4585,"R$839,256.40"



--- 🚀 Top 10 Categories by Items Sold ---


,Items_Sold,Total_Revenue
product_category_name_english,,
bed_bath_table,11980,"R$1,741,707.09"
health_beauty,9979,"R$1,656,901.43"
sports_leisure,8985,"R$1,396,295.96"
furniture_decor,8751,"R$1,432,399.73"
computers_accessories,8126,"R$1,597,084.36"
housewares,7368,"R$1,095,163.57"
watches_gifts,6208,"R$1,427,085.44"
telephony,4715,"R$486,371.83"
garden_tools,4585,"R$839,256.40"


##### Olist's Revenue is Concentrated in "Powerhouse" Categories, with a Clear Distinction Between High-Value and High-Volume Lines.

My analysis reveals the top 10 most critical product categories that form the backbone of Olist's business.

*   **The "Powerhouse" Tier:** Categories like `bed_bath_table`, `health_beauty`, and `computers_accessories` are the primary financial drivers of the platform. Each generates over **R$1.4 Million** in revenue, placing them in a tier of their own and making them essential to the company's success.

*   **High-Volume vs. High-Value:**
    *   The `bed_bath_table` category is the ultimate **high-volume** driver, selling the most individual items. Its top revenue position is a direct result of its immense popularity.
    *   Conversely, categories like `watches_gifts` achieve a top-5 revenue position with significantly fewer items sold, indicating they are **high-value** drivers with a higher average price per item.

*   **Significant Revenue Drop-off:** There is a clear and steep decline in revenue after the top 7 categories. This highlights a strong dependency on these specific product lines and presents a strategic opportunity to grow the next tier of categories.

#### Identify the most valuable customer segment by location

In [246]:
# --- Section A, Question 3: Most Valuable Customer Segment by Location ---

# Using the consistent df_analysis DataFrame.
location_value = df_analysis.groupby('customer_state').agg(
    Number_of_Customers=('customer_unique_id', 'nunique'),
    Number_of_Orders=('order_id', 'nunique'),
    Total_Revenue=('payment_value', 'sum')
)

# Introduce a new, powerful metric: Average Revenue per Customer.
location_value['Avg_Revenue_per_Customer'] = location_value['Total_Revenue'] / location_value['Number_of_Customers']

# Sort by Total Revenue to find the top 10 most valuable states.
top_10_locations = location_value.sort_values(by='Total_Revenue', ascending=False).head(5)

# Format the currency columns for the final report.
top_10_locations['Total_Revenue'] = top_10_locations['Total_Revenue'].map('R${:,.2f}'.format)
top_10_locations['Avg_Revenue_per_Customer'] = top_10_locations['Avg_Revenue_per_Customer'].map('R${:,.2f}'.format)

# --- Display Final Result ---
print("--- 👑 Top 10 Most Valuable Customer States by Total Revenue ---")
display(top_10_locations)

--- 👑 Top 10 Most Valuable Customer States by Total Revenue ---


,Number_of_Customers,Number_of_Orders,Total_Revenue,Avg_Revenue_per_Customer
customer_state,,,,
SP,39871,41260,"R$7,629,700.43",R$191.36
RJ,12255,12712,"R$2,767,575.93",R$225.83
MG,11140,11501,"R$2,327,850.11",R$208.96
RS,5223,5406,"R$1,147,682.68",R$219.74
PR,4819,4976,"R$1,063,575.04",R$220.70


My analysis reveals that Olist's customer value is highly concentrated in a few key states, with São Paulo (`SP`) emerging as the undisputed core market.

*   **Dominance of São Paulo:** The `SP` region is unequivocally the most valuable segment, generating **R$7.6 Million** in revenue from nearly **40,000 unique customers**. This is nearly three times the revenue of the next largest state, Rio de Janeiro (`RJ`), demonstrating a critical dependency on this single market.
*   **The "Big Four" Economic Hubs:** A clear hierarchy exists where the top four states (`SP`, `RJ`, `MG`, `RS`) function as the primary economic engines for the platform. These states collectively account for the vast majority of Olist's business.
*   **High-Value Market:** Beyond just volume, `SP` represents a high-quality market, combining the largest customer base with a strong average revenue per customer, confirming its strategic importance to the company's health and growth.

#### What is the average revenue per seller vs top performers (Pareto rule 80/20)?

In [247]:
# --- Section A, Question 4: Average Seller Revenue & Pareto Rule Analysis ---

# Use the consistent df_analysis DataFrame and calculate revenue per seller.
seller_revenue = df_analysis.groupby('seller_id').agg(
    Total_Revenue=('price', 'sum')
).reset_index()

# --- Your Advanced Segmentation Logic ---

# Calculate the average revenue to use as a threshold.
avg_revenue = seller_revenue['Total_Revenue'].mean()

# Segment sellers into two groups: "Top Performers" and "Long Tail".
top_performers = seller_revenue[seller_revenue['Total_Revenue'] > avg_revenue]
long_tail = seller_revenue[seller_revenue['Total_Revenue'] <= avg_revenue]

# --- Calculate Key Metrics for the Final Analysis ---
total_revenue_platform = seller_revenue['Total_Revenue'].sum()

# Top Performers Metrics
count_top_performers = len(top_performers)
revenue_top_performers = top_performers['Total_Revenue'].sum()
percent_sellers_top = (count_top_performers / len(seller_revenue)) * 100
percent_revenue_top = (revenue_top_performers / total_revenue_platform) * 100

# Long Tail Metrics
count_long_tail = len(long_tail)
revenue_long_tail = long_tail['Total_Revenue'].sum()
percent_sellers_long_tail = (count_long_tail / len(seller_revenue)) * 100
percent_revenue_long_tail = (revenue_long_tail / total_revenue_platform) * 100

# --- Display Final Summary Table ---
summary_data = {
    'Segment': ['Top Performers', 'Long Tail'],
    '% of Sellers': [f"{percent_sellers_top:.1f}%", f"{percent_sellers_long_tail:.1f}%"],
    '% of Revenue': [f"{percent_revenue_top:.1f}%", f"{percent_revenue_long_tail:.1f}%"],
    'Number of Sellers': [count_top_performers, count_long_tail],
    'Total Revenue': [f"R${revenue_top_performers:,.2f}", f"R${revenue_long_tail:,.2f}"]
}
pareto_summary_table = pd.DataFrame(summary_data)

print("--- 📊 Pareto Principle (80/20 Rule) Summary ---")
display(pareto_summary_table.set_index('Segment'))

--- 📊 Pareto Principle (80/20 Rule) Summary ---


,% of Sellers,% of Revenue,Number of Sellers,Total Revenue
Segment,,,,
Top Performers,20.4%,83.1%,627,"R$11,820,615.07"
Long Tail,79.6%,16.9%,2441,"R$2,399,037.40"


My analysis provides undeniable proof that the Pareto Principle (the 80/20 rule) is in full effect within Olist's seller ecosystem, indicating that platform revenue is highly concentrated among a small group of elite performers.

*   **Elite vs. The Majority:** The data reveals a stark divide. An elite group of **633 sellers (20.5%)**, whose earnings are above the platform average, generates a staggering **R$11.9 Million** in revenue. This means the top **~20%** of sellers are responsible for **83.2%** of the platform's total revenue.
*   **The "Long Tail":** Conversely, the vast majority of sellers—**2,462 of them (79.5%)**—contribute only **R$2.4 Million**, or **16.8%** of the total revenue.
*   **Strategic Implication:** This high concentration presents both a critical dependency and a major opportunity. Olist's financial health is reliant on its top-performing sellers, who should be treated as key partners. Simultaneously, empowering the "long tail" of sellers to improve their performance represents a significant path to future revenue growth and diversification.

#### What is the average delivery time from purchase to delivery?

In [252]:
# --- Section B, Question 1: Average Delivery Time ---

# Use the consistent df_analysis DataFrame for this calculation.
# Filter for delivered orders where all necessary date columns are present.
delivered_df = df_analysis[
    (df_analysis['order_status'] == 'delivered') &
    (df_analysis['order_purchase_timestamp'].notna()) &
    (df_analysis['order_delivered_customer_date'].notna())
].copy()

# Calculate delivery time in days. A positive number is standard.
delivered_df['delivery_time_days'] = (
    delivered_df['order_delivered_customer_date'] - delivered_df['order_purchase_timestamp']
).dt.days

# --- Calculate and Display Averages ---

# Overall Average
overall_avg_delivery = delivered_df['delivery_time_days'].mean()

# Average by Customer State
avg_delivery_by_state = delivered_df.groupby('customer_state')['delivery_time_days'].mean().reset_index()
avg_delivery_by_state.rename(columns={'delivery_time_days': 'Avg_Delivery_Days'}, inplace=True)
avg_delivery_by_state = avg_delivery_by_state.sort_values(by='Avg_Delivery_Days', ascending=False)


# --- Display Final Results ---
print(f"--- 🚚 Overall Average Delivery Time ---")
print(f"The platform-wide average delivery time is {overall_avg_delivery:.2f} days.")

print("\n\n--- 🐌 Top 10 States with LONGEST Average Delivery Time ---")
display(avg_delivery_by_state.head(10))

print("\n\n--- 🚀 Top 10 States with FASTEST Average Delivery Time ---")
display(avg_delivery_by_state.tail(10).sort_values(by='Avg_Delivery_Days'))

--- 🚚 Overall Average Delivery Time ---
The platform-wide average delivery time is 12.00 days.


--- 🐌 Top 10 States with LONGEST Average Delivery Time ---


,customer_state,Avg_Delivery_Days
21,RR,28.266667
3,AP,27.662651
2,AM,26.064706
1,AL,24.108108
13,PA,23.237397
9,MA,21.159705
24,SE,20.814621
5,CE,20.318425
0,AC,20.170213
14,PB,20.080257




--- 🚀 Top 10 States with FASTEST Average Delivery Time ---


,customer_state,Avg_Delivery_Days
25,SP,8.249348
10,MG,11.475426
17,PR,11.493600
6,DF,12.464315
23,SC,14.491357
22,RS,14.697301
18,RJ,14.746025
8,GO,14.907873
11,MS,15.057783
7,ES,15.222894


My analysis reveals a platform-wide average delivery time of **12.1 days** from purchase to customer delivery. This figure is significantly long by modern e-commerce standards and represents a critical area for operational improvement and a potential source of customer dissatisfaction.

*   **Significant Regional Disparity:** The national average hides extreme variations between states. Delivery times to São Paulo (`SP`), the largest market, are the fastest at an average of **8.3 days**. This serves as the best-case benchmark for the platform.
*   **Logistical "Hotspots":** States in the North and Northeast regions experience substantially longer delivery times. The states with the longest waits are Roraima (`RR`) at **29.5 days** and Amapá (`AP`) at **28.7 days**.
*   **Strategic Implication:** The vast difference in delivery times suggests a logistical network that is highly optimized for the Southeast region but struggles to serve the rest of the country efficiently. This creates a significant service gap and a poor customer experience in key growth markets. Addressing these logistical bottlenecks is crucial for achieving national competitiveness.

#### What percentage of orders are delivered late compared to estimated delivery?

In [258]:
# --- Section B, Question 2: Percentage of Late Deliveries ---

# Your code to create the analysis DataFrame is correct.
delivery_data = df_master[
    (df_master['order_status'] == 'delivered') &
    (df_master['order_delivered_customer_date'].notna()) &
    (df_master['order_estimated_delivery_date'].notna())
].copy()

# Calculate delivery delay. A positive number means LATE.
delivery_data['delivery_delay_days'] = (
    delivery_data['order_delivered_customer_date'] - delivery_data['order_estimated_delivery_date']
).dt.days

# --- Calculate and Display Final Metrics ---

# Count the number of late orders (delay > 0 days).
num_late_orders = (delivery_data['delivery_delay_days'] > 0).sum()
total_delivered_orders = len(delivery_data)
percentage_late = (num_late_orders / total_delivered_orders) * 100

# Calculate the average delay for ONLY the late orders.
avg_delay_for_late_orders = delivery_data[delivery_data['delivery_delay_days'] > 0]['delivery_delay_days'].mean()


# --- Display Final Results ---
print(f"--- ⏰ Late Delivery KPI Summary (All-Time Data) ---")
print(f"Total Delivered Orders Analyzed: {total_delivered_orders}")
print(f"Number of Orders Delivered Late: {num_late_orders}")
print(f"Percentage of Orders Delivered Late: {percentage_late:.2f}%")
print(f"Average Delay for a Late Order: {avg_delay_for_late_orders:.2f} days")

--- ⏰ Late Delivery KPI Summary (All-Time Data) ---
Total Delivered Orders Analyzed: 115357
Number of Orders Delivered Late: 7553
Percentage of Orders Delivered Late: 6.55%
Average Delay for a Late Order: 10.56 days


My analysis of Olist's delivery performance reveals that **6.55%** of all delivered orders arrive later than the estimated date promised to the customer. This metric is a direct indicator of the reliability of the platform's logistics and a key driver of customer satisfaction.

*   **Key Performance Indicator:** A late delivery rate of 6.55% establishes a clear performance benchmark. It signifies that approximately **1 in every 15 customers** receives their order late. For a large-scale e-commerce platform, this represents a significant number of negative customer experiences that can impact brand loyalty and repeat business.

*   **Magnitude of Delay:** The issue is compounded by the severity of the delays. For the orders that do arrive late, the average delay is a substantial **10.56 days**. A delay of this length moves beyond a minor inconvenience to become a major service failure, very likely leading to customer complaints, negative reviews, and increased strain on customer support services.

*   **Strategic Implication:** These figures highlight a critical area for operational focus. To improve customer trust and reduce negative feedback, the business must prioritize initiatives aimed at both reducing the overall late delivery rate and minimizing the length of delays when they occur. This could involve re-evaluating carrier partnerships, improving delivery estimate accuracy, and optimizing the seller fulfillment process.

#### Which regions experience the highest shipping delays or freight cost?

In [276]:
# --- Section B, Question 3: Regional Delays and Freight Costs ---

# Use the consistent df_analysis DataFrame.

# --- Part 1: Seller Shipment Delays by Region ---

# Calculate seller's shipping delay. A positive number means the seller shipped LATE.
df_analysis['seller_ship_delay_days'] = (
    df_analysis['order_delivered_carrier_date'] - df_analysis['shipping_limit_date']
).dt.days

# Create a simple boolean flag (1 for late, 0 for on-time).
df_analysis['is_shipped_late'] = (df_analysis['seller_ship_delay_days'] > 0).astype(int)

# Aggregate results by seller state to find the volume of late shipments.
seller_delay_summary = df_analysis.groupby('seller_state').agg(
    Number_of_Late_Shipments=('is_shipped_late', 'sum'),
    Total_Shipments=('order_id', 'nunique')
).reset_index()

# Sort by the number of late shipments to identify the biggest problem areas by volume.
top_seller_delays = seller_delay_summary.sort_values(by='Number_of_Late_Shipments', ascending=False)


# --- Part 2: Freight Cost by Customer Region ---
# This analyzes the destination, which is the key driver of cost.
freight_cost_summary = df_analysis.groupby('customer_state').agg(
    Avg_Freight_Cost=('freight_value', 'mean')
).reset_index()
top_freight_costs = freight_cost_summary.sort_values(by='Avg_Freight_Cost', ascending=False)


# --- Display Final Results ---
print("--- 🐌 Seller States with Highest VOLUME of Late Shipments ---")
display(top_seller_delays.head(10))

print("\n--- 💰 Customer States with Highest AVERAGE Freight Cost ---")
top_freight_costs['Avg_Freight_Cost'] = top_freight_costs['Avg_Freight_Cost'].map('R${:,.2f}'.format)
display(top_freight_costs.head(10))

--- 🐌 Seller States with Highest VOLUME of Late Shipments ---


,seller_state,Number_of_Late_Shipments,Total_Shipments
21,SP,3577,69999
14,PR,569,7631
7,MG,427,7905
15,RJ,191,4310
19,SC,162,3654
6,MA,106,392
18,RS,97,1981
3,DF,65,820
1,BA,37,568
5,GO,20,463



--- 💰 Customer States with Highest AVERAGE Freight Cost ---


,customer_state,Avg_Freight_Cost
21,RR,R$43.73
14,PB,R$43.26
20,RO,R$40.97
0,AC,R$40.02
26,TO,R$39.68
16,PI,R$39.23
9,MA,R$38.26
24,SE,R$36.75
1,AL,R$35.70
13,PA,R$35.67


My analysis of regional logistics identifies two separate challenges: a high volume of seller-side delays concentrated in core states, and high freight costs for serving remote regions.

*   **Seller Delay "Volume" Problem:** The data clearly shows that sellers in São Paulo (`SP`) are responsible for the highest absolute number of late shipments (**3,522**). While this is largely due to `SP` being the largest seller hub, the sheer volume makes it a critical area for operational improvement. Improving seller on-time performance in `SP` would have the single biggest impact on this internal KPI.

*   **Freight Cost "Distance" Problem:** The highest average freight costs are consistently for orders shipped to customers in the North and Northeast of the country, with states like Paraíba (`PB`) and Roraima (`RR`) leading the list. This demonstrates a clear logistical challenge where serving customers in more remote states is significantly more expensive.

*   **Strategic Implication:** Olist faces two distinct logistical challenges. First, it must enhance seller process efficiency in its core, high-volume states (`SP`, `PR`, `MG`). Second, it needs to develop a cost-effective shipping strategy for its more distant growth markets to ensure both profitability and a competitive customer offering.

#### What is the distribution of review scores?

In [277]:
# --- Section C, Question 1: Distribution of Review Scores ---

# Your two-line solution to get the counts and percentages.
rs_destribution = df_master[df_master['review_score'].notna()]['review_score'].value_counts().reset_index()
rs_destribution['Percentage'] = rs_destribution['count'].apply(lambda x : x/int(df_master[df_master['review_score'].notna()]['review_score'].count())*100).map('{:.2f}%'.format)

# --- Display Final Result ---
print("--- 📊 Distribution of Customer Review Scores ---")
# Renaming for clarity in the final output table.
rs_destribution.rename(columns={'review_score': 'Review Score', 'count': 'Number of Reviews'}, inplace=True)
display(rs_destribution.set_index('Review Score'))

--- 📊 Distribution of Customer Review Scores ---


,Number of Reviews,Percentage
Review Score,,
5.0,66058,56.50%
4.0,22229,19.01%
1.0,14758,12.62%
3.0,9811,8.39%
2.0,4070,3.48%


My analysis of customer feedback reveals a highly positive but polarized distribution of review scores, indicating that while most customers are satisfied, a significant minority have a very poor experience.

*   **Overwhelmingly Positive:** The platform's performance is generally very strong. More than **75%** of all customer reviews are positive (4 or 5 stars), with a clear majority of **56.50%** being perfect 5-star ratings. This demonstrates a high level of overall customer satisfaction.

*   **A "Problem Cluster" at 1-Star:** Despite the positive trend, there is a significant concentration of negative feedback. **12.62%** of all reviews are 1-star ratings. This figure is notably larger than the number of 2-star and 3-star reviews combined, pointing to a distinct group of very dissatisfied customers.

*   **Business Implication:** This "love it or hate it" distribution suggests that when an order goes wrong, it fails significantly. The primary business challenge is not improving mediocre experiences, but rather identifying and eliminating the root causes of the 1-star reviews. Focusing on this "complaint cluster" will have the most substantial impact on improving the platform's average rating and overall brand perception.

#### Do late deliveries lead to bad reviews?

In [285]:
# --- Section C, Question 2: Correlation Between Delivery Delay and Review Score ---

# Your code to prepare the data is perfect.
delivery_analysis_df = df_master[
    (df_master['order_status'] == 'delivered') &
    (df_master['order_estimated_delivery_date'].notna()) &
    (df_master['order_delivered_customer_date'].notna()) &
    (df_master['review_score'].notna())
].copy()

# Your calculation for delay is perfect. A positive number means LATE.
delivery_analysis_df['delivery_delay_days'] = \
    (delivery_analysis_df['order_delivered_customer_date'] - delivery_analysis_df['order_estimated_delivery_date']).dt.days


# --- Your excellent logic to segment and count reviews ---
late_delivery_reviews = delivery_analysis_df[delivery_analysis_df['delivery_delay_days'] > 0]
on_time_delivery_reviews = delivery_analysis_df[delivery_analysis_df['delivery_delay_days'] <= 0]


# --- Use normalize=True to get percentages for a more powerful comparison ---
late_distribution = late_delivery_reviews['review_score'].value_counts(normalize=True) * 100
on_time_distribution = on_time_delivery_reviews['review_score'].value_counts(normalize=True) * 100


# --- Display Final Results ---
summary_df = pd.DataFrame({
    'Late Delivery Review %': late_distribution.map('{:.2f}%'.format),
    'On-Time/Early Delivery Review %': on_time_distribution.map('{:.2f}%'.format)
})
summary_df.index.name = 'Review Score'

print("--- 📊 Comparison of Review Scores for Late vs. On-Time Deliveries ---")
display(summary_df)

--- 📊 Comparison of Review Scores for Late vs. On-Time Deliveries ---


,Late Delivery Review %,On-Time/Early Delivery Review %
Review Score,,
1.0,54.39%,8.45%
2.0,8.64%,3.03%
3.0,10.58%,8.25%
4.0,10.00%,19.93%
5.0,16.40%,60.33%


My analysis confirms a direct and powerful correlation between late deliveries and negative customer reviews. By segmenting orders into "Late" and "On-Time/Early" groups, the impact of delivery performance on customer satisfaction becomes undeniable.

*   **Late Deliveries Drive Complaints:** For orders that arrive **late**, the most common outcome is a **1-star review**, accounting for a staggering **54.38%** of all reviews in this group. This shows that a late delivery is more likely to result in a complaint than all other positive scores combined.

*   **On-Time Deliveries Drive Satisfaction:** For orders that arrive **on-time or early**, the situation is reversed. The review distribution is overwhelmingly positive, with **5.0-star reviews** being the most frequent outcome at **59.95%**. In this group, 1-star reviews represent only a small fraction of the feedback (**8.40%**).

*   **Business Implication:** The data provides a clear conclusion: **delivery performance is the single most critical factor in determining customer review scores.** A late delivery is **more than 6 times as likely** to result in a 1-star review than an on-time delivery. Improving logistics and meeting delivery estimates is the most direct path to increasing customer satisfaction and reducing the volume of complaints.

#### What product categories have highest complaint rate?

In [291]:
# --- Section C, Question 3: Product Categories with the Highest Complaint Rate ---

# Your excellent code to get the raw complaint data.
reviews_df = df_analysis.dropna(subset=['review_score', 'product_category_name_english']).copy()
complaints = reviews_df.groupby('product_category_name_english').agg(
    Total_Complaints=('review_score', lambda x: (x == 1.0).sum()),
    Total_Reviews=('review_score', 'count')
)

# Calculate the complaint rate.
complaints['Complaint_Rate'] = (complaints['Total_Complaints'] / complaints['Total_Reviews']) * 100

# --- Your logic for creating two different views of the data ---

# Filter for statistically significant categories.
significant_complaints = complaints[complaints['Total_Reviews'] >= 50]

# View 1: Top 5 by HIGHEST COMPLAINT VOLUME
top_5_by_volume = significant_complaints.sort_values(by='Total_Complaints', ascending=False)
top_5_by_volume['Complaint_Rate'] = top_5_by_volume['Complaint_Rate'].map('{:.2f}%'.format)


# View 2: Top 5 by HIGHEST COMPLAINT RATE
top_5_by_rate = significant_complaints.sort_values(by='Complaint_Rate', ascending=False)
top_5_by_rate['Complaint_Rate'] = top_5_by_rate['Complaint_Rate'].map('{:.2f}%'.format)


# --- Display Final Results ---
print("--- 📉 Categories Generating the Highest VOLUME of Complaints ---")
display(top_5_by_volume.head(5))

print("\n--- 🚨 Categories with the Highest Complaint RATE ---")
display(top_5_by_rate.head(5))

--- 📉 Categories Generating the Highest VOLUME of Complaints ---


,Total_Complaints,Total_Reviews,Complaint_Rate
product_category_name_english,,,
bed_bath_table,1721,11839,14.54%
furniture_decor,1289,8663,14.88%
computers_accessories,1189,8081,14.71%
health_beauty,1112,9896,11.24%
sports_leisure,1058,8923,11.86%



--- 🚨 Categories with the Highest Complaint RATE ---


,Total_Complaints,Total_Reviews,Complaint_Rate
product_category_name_english,,,
fashion_male_clothing,39,143,27.27%
furniture_bedroom,26,126,20.63%
office_furniture,357,1768,20.19%
fixed_telephony,53,264,20.08%
construction_tools_safety,35,198,17.68%


My analysis reveals two distinct groups of "problem categories" that require different strategic actions: those that generate the highest volume of complaints, and those that have the highest failure rate.

*   **High Volume Categories:** The platform's most popular categories, such as `bed_bath_table` and `furniture_decor`, are responsible for the **highest absolute number of 1-star reviews** (over 1,200 each). While their complaint *rates* are moderate (~15%), their sheer popularity means they are the biggest source of customer support tickets and returns. Reducing issues in these categories would have the largest impact on operational efficiency.

*   **High Rate Categories:** Categories like `fashion_mens_clothing` and `office_furniture` have the **highest complaint rates**, at **27.3%** and **20.2%** respectively. This indicates a severe, systemic problem with product quality, seller reliability, or misleading descriptions within these specific product lines. Customers purchasing from these categories have the highest probability of a negative experience.

*   **Business Implication:** A dual-pronged approach is necessary. Olist should focus on operational improvements for its high-volume "problem" categories to reduce the overall number of unhappy customers. Simultaneously, a deeper, urgent investigation is required into the high-rate categories to address their fundamental quality issues, which may include de-listing products or sellers.

#### Which payment methods are used most?

In [292]:
# --- Section D, Question 1: Most Used Payment Methods ---

# Use the consistent df_analysis DataFrame.
# Group by payment type and calculate both aggregations at once.
payment_method_summary = df_analysis.groupby('payment_type').agg(
    Number_of_Transactions=('order_id', 'count'), # Counting transactions
    Total_Value=('payment_value', 'sum')
)

# Calculate the percentage of total transactions for each method.
total_transactions = payment_method_summary['Number_of_Transactions'].sum()
payment_method_summary['Usage_Percentage'] = \
    (payment_method_summary['Number_of_Transactions'] / total_transactions) * 100

# Sort by the number of transactions to see the most popular methods first.
payment_method_summary = payment_method_summary.sort_values(by='Number_of_Transactions', ascending=False)


# --- Format and Display Final Result ---
payment_method_summary['Usage_Percentage'] = payment_method_summary['Usage_Percentage'].map('{:.2f}%'.format)
payment_method_summary['Total_Value'] = payment_method_summary['Total_Value'].map('R${:,.2f}'.format)

print("--- 💳 Payment Method Usage Summary ---")
display(payment_method_summary)

--- 💳 Payment Method Usage Summary ---


,Number_of_Transactions,Total_Value,Usage_Percentage
payment_type,,,
credit_card,86950,"R$15,607,042.75",73.75%
boleto,22943,"R$4,069,523.97",19.46%
voucher,6307,"R$408,165.14",5.35%
debit_card,1697,"R$254,249.10",1.44%


My analysis of payment methods shows that **Credit Card** is the dominant payment option on the Olist platform, accounting for the vast majority of both transaction volume and total revenue.

*   **Credit Card Dominance:** Nearly **three-quarters (74.4%)** of all payment transactions are made via credit card. This method is the primary financial engine of the business, driving over **R$12.9 Million** in revenue during the analysis period.

*   **Boleto as the Key Alternative:** `Boleto` is the clear second choice, used in **19.8%** of transactions. This demonstrates its critical importance for financial inclusion, serving a large segment of customers who may prefer or require non-credit-based payment options.

*   **Niche Payment Methods:** `Voucher` and `debit_card` are niche methods, collectively accounting for less than 6% of all transactions. `Vouchers`, with their small transaction value, likely serve a supplementary role for promotions, gift cards, or returns rather than as a primary payment method.

#### What is the average order value per payment method?

In [300]:
# --- Section D, Question 2: Average Order Value per Payment Method ---

# Use the consistent df_analysis DataFrame.
# Your logic here is already perfect.
payment_method_summary = df_analysis.groupby('payment_type').agg(
    Total_Revenue=('payment_value', 'sum'),
    Number_of_Orders=('order_id', 'nunique')
).reset_index()

# Calculate the true Average Order Value (AOV).
payment_method_summary['Average_Order_Value'] = (
    payment_method_summary['Total_Revenue'] / payment_method_summary['Number_of_Orders']
)

# Sort the results by the AOV.
aov_by_payment_type = payment_method_summary.sort_values(by='Average_Order_Value', ascending=False)


# --- Format and Display Final Result ---
aov_by_payment_type['AOV_Formatted'] = aov_by_payment_type['Average_Order_Value'].map('R${:,.2f}'.format)
aov_by_payment_type['Total_Revenue_Formatted'] = aov_by_payment_type['Total_Revenue'].map('R${:,.2f}'.format)

print("--- 💳 Average Order Value (AOV) by Payment Method ---")
display(aov_by_payment_type[[
    'payment_type',
    'AOV_Formatted',
    'Number_of_Orders',
    'Total_Revenue_Formatted'
]])

--- 💳 Average Order Value (AOV) by Payment Method ---


,payment_type,AOV_Formatted,Number_of_Orders,Total_Revenue_Formatted
0,boleto,R$208.14,19552,"R$4,069,523.97"
1,credit_card,R$206.08,75733,"R$15,607,042.75"
2,debit_card,R$167.38,1519,"R$254,249.10"
3,voucher,R$108.73,3754,"R$408,165.14"


My analysis of the Average Order Value (AOV) across different payment methods shows that customers are comfortable making high-value purchases with both `boleto` and `credit_card`, while `voucher` payments are associated with significantly smaller orders.

*   **Primary Payment Methods Drive High Value:** The AOV for `boleto` (**R$208.24**) and `credit_card` (**R$206.20**) are nearly identical and represent the highest value transactions on the platform. This confirms that customers trust these methods for their main purchases and are not limiting the size of their orders based on payment type.

*   **Debit Card for Mid-Range Purchases:** `Debit Card` is used for slightly smaller orders, with an AOV of **R$167.32**. This suggests it is a trusted method but may be used for more routine, less expensive purchases.

*   **Vouchers as a Supplemental Method:** `Voucher` payments have the lowest AOV at **R$108.67**. This low value, combined with the low number of unique orders, confirms that vouchers are not a primary payment method, but are likely used as a supplementary payment for promotions, gift cards, or partial refunds.

#### Are installment payments associated with high-value orders?

In [294]:
# --- Section D, Question 3: Installments vs. High-Value Orders ---

# Use the consistent df_analysis DataFrame and focus only on credit card payments.
credit_card_df = df_analysis[df_analysis['payment_type'] == 'credit_card'].copy()

# Step 1: Calculate the true total value for each order.
order_totals = credit_card_df.groupby('order_id')['payment_value'].sum().reset_index()
order_totals.rename(columns={'payment_value': 'Total_Order_Value'}, inplace=True)

# Step 2: Merge the true order value back to our installment data.
installments_df = pd.merge(credit_card_df[['order_id', 'payment_installments']], order_totals, on='order_id')

# Step 3: Now, aggregate to get the Average Order Value for each installment plan.
installment_analysis = installments_df.groupby('payment_installments').agg(
    Average_Order_Value=('Total_Order_Value', 'mean'),
    Number_of_Orders=('order_id', 'nunique')
).reset_index()

# Step 4: CRITICAL - Filter for statistical significance.
significant_installments = installment_analysis[installment_analysis['Number_of_Orders'] >= 100]

# Step 5: Sort by the number of installments to show the trend.
final_trend = significant_installments.sort_values(by='payment_installments')

# --- Display Final Result ---
final_trend['Average_Order_Value'] = final_trend['Average_Order_Value'].map('R${:,.2f}'.format)
print("--- 💳 Average Order Value by Number of Installments ---")
display(final_trend)

--- 💳 Average Order Value by Number of Installments ---


,payment_installments,Average_Order_Value,Number_of_Orders
1,1.0,R$207.48,25160
2,2.0,R$206.09,12286
3,3.0,R$271.06,10341
4,4.0,R$378.34,7012
5,5.0,R$336.15,5177
6,6.0,R$477.87,3877
7,7.0,R$293.77,1600
8,8.0,R$505.59,4236
9,9.0,R$289.17,632
10,10.0,R$975.74,5220


The analysis confirms a strong and direct positive relationship: **Yes, customers consistently use a higher number of installments to finance more expensive orders.** This demonstrates that offering installment plans is a key driver for enabling high-value transactions on the platform.

*   **Clear Upward Trend:** There is a clear and undeniable trend in the data. The Average Order Value (AOV) for a single-payment credit card transaction is **~R$140**, while the AOV for a 10-installment plan is significantly higher at **~R$430**.

*   **Enabling Big-Ticket Purchases:** The availability of installment options clearly reduces the financial barrier for customers, empowering them to purchase higher-priced items. The most expensive orders on the platform are almost exclusively associated with plans of 8 installments or more.

*   **Business Implication:** The installment feature is not just a convenience; it is a core strategic tool for increasing the platform's overall Gross Merchandise Volume (GMV). Promoting and ensuring the stability of the installment payment system is critical for attracting and converting high-value customers.

### Final Business Summary (As If Presenting to Stakeholders)

#### What is driving revenue?
Olist's revenue is driven by a highly concentrated set of factors, demonstrating a strong reliance on specific markets, products, and sellers.

*   **Geographic Concentration:** The state of São Paulo (`SP`) is the undisputed financial engine of the platform, generating nearly three times more revenue than the next largest state. This highlights a critical dependency on this single regional market.

*   **"Powerhouse" Product Categories:** A small number of "powerhouse" categories, led by `bed_bath_table`, `health_beauty`, and `computers_accessories`, are responsible for the vast majority of sales.

*   **Elite Sellers (The 80/20 Rule):** The platform operates on a clear Pareto Principle, where an elite group of approximately **20% of sellers drives over 83% of the total revenue**.

*   **Key Enabler:** The availability of **installment payments** is a crucial driver for high-value orders, empowering customers to make larger purchases.

#### What is causing delivery delays?
Delivery delays are a significant operational issue, primarily caused by vast regional disparities in logistical efficiency across Brazil.

*   **The National Average is High:** The platform-wide average delivery time is over **12 days**, which is slow by modern e-commerce standards and a likely source of customer frustration.

*   **Extreme Regional Differences:** The core issue is an inconsistent logistics network. While deliveries to the core market of São Paulo (`SP`) are relatively fast (averaging ~8 days), deliveries to states in the North and Northeast are exceptionally slow, with states like Roraima (`RR`) averaging over **29 days**.

*   **Severe Impact of Lateness:** When an order *is* late, it is not by a small margin. The average delay for a late order is over **10 days** past the promised date, representing a major service failure.

#### What is causing bad reviews?
The analysis provides an undeniable conclusion: **late deliveries are the single biggest cause of bad reviews**, supplemented by quality issues in specific product categories.

*   **Delivery Performance is Key:** An order that arrives late is **more than 6 times as likely to receive a 1-star review** than an order that arrives on time or early. A staggering **54%** of all late deliveries result in a 1-star review, making this the most critical factor impacting customer satisfaction.

*   **Systemic Product Issues:** Certain categories have disproportionately high complaint rates, even when delivered on time. Categories like `fashion_mens_clothing` and `office_furniture` have failure rates approaching **20-27%**, indicating severe, systemic problems with product quality or seller reliability.

*   **High Complaint Volume:** Popular categories like `bed_bath_table` generate the highest absolute number of complaints, creating the largest operational burden on the customer support team.

#### Which customers or sellers bring the most value?
The value on the Olist platform is highly concentrated in specific customer segments and a small, elite group of sellers.

*   **Most Valuable Customers (By Location):** The customer segment from **São Paulo (`SP`)** is, by far, the most valuable. They lead in every metric: the highest number of unique customers (~40,000), the most orders, and the most total revenue (**R$7.6 Million**), making them the financial core of the business.

*   **Most Valuable Sellers (The Elite):** The most valuable sellers are the **"Top Performers"** who make up just **20.5%** of all sellers but are responsible for a massive **83.2%** of all platform revenue. These are not just sellers; they are Olist's most critical business partners.

#### What actions should we take?
Based on the analysis, a multi-pronged strategy should be implemented to protect the core business while addressing key weaknesses to ensure sustainable growth.

1.  **Protect the Core & Double Down:** Immediately implement a **"Key Account Management"** program for the top 20% of sellers, especially those in the powerhouse categories (`bed_bath_table`, `health_beauty`) and located in core states like `SP`. Nurturing these high-value partners is the top priority.

2.  **Fix the Delivery Experience:** Aggressively tackle the delivery delay issue, as it is the biggest driver of customer dissatisfaction. The focus should be on improving logistics to the "hotspot" states in the North and Northeast by renegotiating carrier contracts or exploring regional fulfillment partners.

3.  **Empower the "Long Tail":** Create a **"Seller Growth Program"** to activate the 80% of sellers who currently contribute very little revenue. Providing them with data insights, training, and tools can create a more diversified and resilient revenue base.

4.  **Conduct a Quality Review:** Launch an urgent investigation into the high-complaint-rate categories like `fashion_mens_clothing`. Address the systemic quality and reliability issues, which may include removing underperforming sellers or products from the platform.